In [25]:
import sys
sys.path.append("./../../src")
import pandas as pd
import numpy as np

import torch
from torch import nn, optim

In [26]:
from vae_with_dataloader import VAE, VAETrainerWithDataLoader
from icd_analysis_helper import ICDAnalysisHelper
from dataloader import PatientICDSparseVanillaDataset
from visualizer_helper import Visualizer
import pickle

from config_reader import Config
import os
import math
from sklearn.model_selection import train_test_split

In [27]:
###Load Configuration file
config = Config('./config.ini')
experiment_name=config.experiment_name
config.__dict__

{'patient_icd_path': '/Users/andrew/Documents/meng/spring/PATIENT_ICD_BINARY_SPARSE_CSR.p',
 'icd9codes_path': '/data1/andrew/meng/mixehr/data/Mimic/mimic-iii-clinical-database-1.4/D_ICD_DIAGNOSES.csv.gz',
 'subject_ids_path': '.',
 'experiment_name': '200213_patient_clusters_25_latent_dim_vae_with_larger_decoder',
 'encoder_dim': [(250, 500), (500, 250), (250, 100)],
 'latent_dim': 25,
 'decoder_dim': [(250, 500), (500, 250), (250, 100)],
 'use_relu_encoder': True,
 'use_relu_decoder': True,
 'kld_beta': 1.0}

In [28]:
###Load Data
patient_icd_dataset = PatientICDSparseVanillaDataset(csr_data_path = "./../../data/PATIENT_ICD_BINARY_SPARSE_CSR.p")

Loaded CSR Dataset w/ dim (46520, 6984)


In [29]:
# icd9codes = pd.read_csv(config.icd9codes_path)
# icd_analyzer = ICDAnalysisHelper(icd9codes_df = icd9codes, patient_icd_df = patient_icd_df)
#icd_analyzer.lookup_icds(icd9codes, ["4019", "41401"])
visualizer = Visualizer()

In [30]:
###Load Model
model = VAE(
    feature_dim = patient_icd_dataset.get_feat_dim(), 
    encoder_dim = config.encoder_dim,
    latent_dim = config.latent_dim,
    decoder_dim = config.decoder_dim,
    use_relu_encoder= config.use_relu_encoder,
    use_relu_decoder= config.use_relu_decoder
)

optimizer = optim.Adam(model.parameters(), lr=0.001)

###Load Trainer
trainer = VAETrainerWithDataLoader(
    model=model, 
    optimizer=optimizer,
    experiment_name=experiment_name,
    kld_beta=config.kld_beta
)

In [31]:
training_params = {'batch_size':64, 'shuffle': True}
training_generator = torch.utils.data.DataLoader(patient_icd_dataset, **training_params)

In [33]:
###Train Model
trainer.train(
    training_generator=training_generator,
    epochs=80,
    batch_size=40,
    save_model_interval=5,
    clip_gradients=False
)

  0%|          | 0/727 [00:00<?, ?it/s]

Training with KLD Beta weight of 1.0


  0%|          | 0/727 [00:00<?, ?it/s]


RuntimeError: sparse tensors do not have is_contiguous

In [ ]:
trainer.plot_elbo()
trainer.plot_bce()
trainer.plot_kld()

In [ ]:
###Load pre-trained model
epoch = 40
model.load_state_dict(torch.load("./VAE_exp_{}_epoch_{}.pkl".format(experiment_name, epoch)))
trainer.model = model

In [ ]:
###Encode data
encoding_params = {'batch_size':64, 'shuffle': False}
encoding_generator = torch.utils.data.DataLoader(patient_icd_dataset, **encoding_params)

latent, means, var = trainer.encode_data(encoding_generator)
pickle.dump(latent, open("latent_epochs_{}.p".format(epoch), 'wb'))
pickle.dump(means, open("means_epochs_{}.p".format(epoch), 'wb'))
pickle.dump(var, open("vars_epochs_{}.p".format(epoch), 'wb'))

In [ ]:
###Get UMAP representations
X_umap = visualizer.umap_embedding(latent.cpu().detach().numpy(), n_components=3)
mean_umap = visualizer.umap_embedding(means.cpu().detach().numpy(), n_components=3)
var_umap = visualizer.umap_embedding(var.cpu().detach().numpy(), n_components=3)

pickle.dump(X_umap, open("X_umap_3d.p", 'wb'))
pickle.dump(mean_umap, open("mean_umap_3d.p", 'wb'))
pickle.dump(var_umap, open("var_umap_3d.p", 'wb'))

In [ ]:
###Additional Visualizations
baby_keywords = ['congenital', 'infant', 'newborn', 'neonatal', 'born', 'birth']
heart_keywords = ['heart', 'atrial', 'coronary', 'hypertension', 'vascular']


heart_patient_idxs_tr = icd_analyzer.get_patients_idxs_with_disease_keywords(patient_df=train_df, 
                                                                             substrings=heart_keywords, 
                                                                             case_sensitive=False
                                                                            )
baby_patient_idxs_tr = icd_analyzer.get_patients_idxs_with_disease_keywords(patient_df=train_df,
                                                                            substrings=baby_keywords,
                                                                            case_sensitive=False
                                                                           )

baby_heart_colors_tr = np.array([0 for i in range(X_umap_tr.shape[0])])
baby_heart_colors_tr[heart_patient_idxs_tr] = 100
baby_heart_colors_tr[baby_patient_idxs_tr] = 50

visualizer.plot2d(
    X=X_umap_tr, 
    filename="Patient_Clusters_exp_{}_epoch_{}{}".format(experiment_name, epoch, "_baby_heart_umap_train"), 
    colors=baby_heart_colors_tr)


heart_patient_idxs_val = icd_analyzer.get_patients_idxs_with_disease_keywords(patient_df=val_df, 
                                                                              substrings=heart_keywords,
                                                                              case_sensitive=False)
baby_patient_idxs_val = icd_analyzer.get_patients_idxs_with_disease_keywords(patient_df=val_df,
                                                                             substrings=baby_keywords,
                                                                             case_sensitive=False)

baby_heart_colors_val = np.array([0 for i in range(X_umap_val.shape[0])])
baby_heart_colors_val[heart_patient_idxs_val] = 100
baby_heart_colors_val[baby_patient_idxs_val] = 50
visualizer.plot2d(
    X=X_umap_val, 
    filename="Patient_Clusters_exp_{}_epoch_{}{}".format(experiment_name, epoch, "_baby_heart_umap_val"), 
    colors=baby_heart_colors_val)

In [ ]:
X_umap_3d_tr = visualizer.umap_embedding(latent_train.cpu().detach().numpy(), n_components=3)

visualizer.plot3d(
    X=X_umap_3d_tr, 
    filename="Patient_Clusters_exp_{}_epoch_{}{}".format(experiment_name, epoch, "_heart_umap_3D_train"), 
    colors=baby_heart_colors_tr,
)

In [ ]:
###Plot UMAP representations
heart_patient_idxs = icd_analyzer.get_patients_idxs_with_disease_keywords(substrings=heart_keywords, case_sensitive=False)
heart_colors = np.array([0 for i in range(X_umap.shape[0])])
heart_colors[heart_patient_idxs] = 100

visualizer.plot2d(
    X=X_umap_tr, 
    filename="Patient_Clusters_exp_{}_epoch_{}{}".format(experiment_name, epoch, "_heart_umap_train"), 
    colors=heart_colors,
)
visualizer.plot2d(
    X=X_umap_means_tr, 
    filename="Patient_Clusters_exp_{}_epoch_{}{}".format(experiment_name, epoch, "_heart_umap_means_train"), 
    colors=heart_colors,
)
visualizer.plot2d(
    X=X_umap_vars_tr, 
    filename="Patient_Clusters_exp_{}_epoch_{}{}".format(experiment_name, epoch, "_heart_umap_vars_train"), 
    colors=heart_colors,
)